Import stuff

In [1]:
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime, timedelta
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import csv

In [2]:
ip_address = "YOUR IP"
client = MongoClient(f"{ip_address}", 27017)
db = client.fit3182_assignment_db
collection = db.climate_hotspot
query = collection.aggregate([{"$project" : {"_id" : 0, "date" : 1}}, {"$sort" : {"date" : -1}}, {"$limit" : 1}])
start_date = query.__next__()["date"]

In [3]:
start_date

datetime.datetime(2023, 1, 1, 0, 0)

Now we can load the data from hotspot_TERRA_streaming.csv

In [4]:
terra_data = []
with open("hotspot_TERRA_streaming.csv", "r") as terra_file:
    for terra in csv.DictReader(terra_file):
        ##change some data types
        terra["latitude"] = float(terra["latitude"])
        terra["longitude"] = float(terra["longitude"])
        terra["confidence"] = int(terra["confidence"])
        terra["surface_temperature_celcius"] = int(terra["surface_temperature_celcius"])
        terra_data.append(terra)

Now to see a preview of the data to pass to the producer

In [5]:
data = {"datetime" : str(start_date.strftime("%d/%m/%Y %H:%M:%S")), "terra" : terra_data[random.randrange(0,len(terra_data))]}

In [6]:
data

{'datetime': '01/01/2023 00:00:00',
 'terra': {'latitude': -36.0844,
  'longitude': 141.4934,
  'confidence': 81,
  'surface_temperature_celcius': 71}}

Now setup the kafka producer

In [7]:
host_ip = "YOUR IP"

def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [8]:
latest_date = datetime(start_date.year, start_date.month, start_date.day)
topic = 'Terra'

print('Publishing records..')
producer = connect_kafka_producer()

while True:
    data = {"datetime" : str(latest_date.strftime("%d/%m/%Y %H:%M:%S")), "terra" : terra_data[random.randrange(0,len(terra_data))]}
    publish_message(producer, topic, 'jsondata', data)
    latest_date = latest_date + timedelta(hours=4, minutes = 48) ##2 seconds is 4 hours 48 minutes
    sleep(2)

Publishing records..
Message published successfully. Data: {'datetime': '01/01/2023 00:00:00', 'terra': {'latitude': -37.6439, 'longitude': 142.913, 'confidence': 93, 'surface_temperature_celcius': 83}}
Message published successfully. Data: {'datetime': '01/01/2023 04:48:00', 'terra': {'latitude': -37.1248, 'longitude': 141.8795, 'confidence': 71, 'surface_temperature_celcius': 52}}
Message published successfully. Data: {'datetime': '01/01/2023 09:36:00', 'terra': {'latitude': -36.6672, 'longitude': 143.8631, 'confidence': 79, 'surface_temperature_celcius': 52}}
Message published successfully. Data: {'datetime': '01/01/2023 14:24:00', 'terra': {'latitude': -37.7851, 'longitude': 143.3753, 'confidence': 83, 'surface_temperature_celcius': 64}}
Message published successfully. Data: {'datetime': '01/01/2023 19:12:00', 'terra': {'latitude': -36.3735, 'longitude': 147.3154, 'confidence': 86, 'surface_temperature_celcius': 61}}
Message published successfully. Data: {'datetime': '02/01/2023 00

Message published successfully. Data: {'datetime': '10/01/2023 00:00:00', 'terra': {'latitude': -36.1256, 'longitude': 145.2444, 'confidence': 77, 'surface_temperature_celcius': 53}}
Message published successfully. Data: {'datetime': '10/01/2023 04:48:00', 'terra': {'latitude': -37.7457, 'longitude': 142.9428, 'confidence': 80, 'surface_temperature_celcius': 53}}
Message published successfully. Data: {'datetime': '10/01/2023 09:36:00', 'terra': {'latitude': -37.441, 'longitude': 142.9966, 'confidence': 65, 'surface_temperature_celcius': 42}}
Message published successfully. Data: {'datetime': '10/01/2023 14:24:00', 'terra': {'latitude': -36.031, 'longitude': 145.6824, 'confidence': 91, 'surface_temperature_celcius': 69}}
Message published successfully. Data: {'datetime': '10/01/2023 19:12:00', 'terra': {'latitude': -38.0219, 'longitude': 141.2238, 'confidence': 76, 'surface_temperature_celcius': 62}}
Message published successfully. Data: {'datetime': '11/01/2023 00:00:00', 'terra': {'la

Message published successfully. Data: {'datetime': '19/01/2023 00:00:00', 'terra': {'latitude': -37.227, 'longitude': 141.151, 'confidence': 59, 'surface_temperature_celcius': 35}}
Message published successfully. Data: {'datetime': '19/01/2023 04:48:00', 'terra': {'latitude': -36.3839, 'longitude': 141.3032, 'confidence': 83, 'surface_temperature_celcius': 56}}
Message published successfully. Data: {'datetime': '19/01/2023 09:36:00', 'terra': {'latitude': -36.1021, 'longitude': 145.5294, 'confidence': 67, 'surface_temperature_celcius': 47}}
Message published successfully. Data: {'datetime': '19/01/2023 14:24:00', 'terra': {'latitude': -35.2068, 'longitude': 143.2912, 'confidence': 68, 'surface_temperature_celcius': 44}}
Message published successfully. Data: {'datetime': '19/01/2023 19:12:00', 'terra': {'latitude': -36.993, 'longitude': 141.2317, 'confidence': 67, 'surface_temperature_celcius': 43}}
Message published successfully. Data: {'datetime': '20/01/2023 00:00:00', 'terra': {'lat

Message published successfully. Data: {'datetime': '28/01/2023 00:00:00', 'terra': {'latitude': -37.437, 'longitude': 148.104, 'confidence': 99, 'surface_temperature_celcius': 61}}
Message published successfully. Data: {'datetime': '28/01/2023 04:48:00', 'terra': {'latitude': -37.8567, 'longitude': 147.1576, 'confidence': 62, 'surface_temperature_celcius': 48}}
Message published successfully. Data: {'datetime': '28/01/2023 09:36:00', 'terra': {'latitude': -36.801, 'longitude': 144.0114, 'confidence': 70, 'surface_temperature_celcius': 45}}
Message published successfully. Data: {'datetime': '28/01/2023 14:24:00', 'terra': {'latitude': -36.6152, 'longitude': 142.4911, 'confidence': 84, 'surface_temperature_celcius': 58}}
Message published successfully. Data: {'datetime': '28/01/2023 19:12:00', 'terra': {'latitude': -36.7217, 'longitude': 141.6378, 'confidence': 83, 'surface_temperature_celcius': 56}}
Message published successfully. Data: {'datetime': '29/01/2023 00:00:00', 'terra': {'lat

Message published successfully. Data: {'datetime': '06/02/2023 00:00:00', 'terra': {'latitude': -36.834, 'longitude': 141.9115, 'confidence': 71, 'surface_temperature_celcius': 46}}
Message published successfully. Data: {'datetime': '06/02/2023 04:48:00', 'terra': {'latitude': -35.364, 'longitude': 141.063, 'confidence': 72, 'surface_temperature_celcius': 54}}
Message published successfully. Data: {'datetime': '06/02/2023 09:36:00', 'terra': {'latitude': -36.9034, 'longitude': 141.0013, 'confidence': 100, 'surface_temperature_celcius': 95}}
Message published successfully. Data: {'datetime': '06/02/2023 14:24:00', 'terra': {'latitude': -36.6892, 'longitude': 145.5397, 'confidence': 75, 'surface_temperature_celcius': 49}}
Message published successfully. Data: {'datetime': '06/02/2023 19:12:00', 'terra': {'latitude': -37.7168, 'longitude': 149.1369, 'confidence': 77, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '07/02/2023 00:00:00', 'terra': {'la

Message published successfully. Data: {'datetime': '15/02/2023 00:00:00', 'terra': {'latitude': -37.5949, 'longitude': 142.6857, 'confidence': 100, 'surface_temperature_celcius': 98}}
Message published successfully. Data: {'datetime': '15/02/2023 04:48:00', 'terra': {'latitude': -36.1376, 'longitude': 145.84, 'confidence': 76, 'surface_temperature_celcius': 55}}
Message published successfully. Data: {'datetime': '15/02/2023 09:36:00', 'terra': {'latitude': -37.4437, 'longitude': 143.4924, 'confidence': 72, 'surface_temperature_celcius': 46}}
Message published successfully. Data: {'datetime': '15/02/2023 14:24:00', 'terra': {'latitude': -36.0144, 'longitude': 141.7008, 'confidence': 51, 'surface_temperature_celcius': 38}}
Message published successfully. Data: {'datetime': '15/02/2023 19:12:00', 'terra': {'latitude': -36.433, 'longitude': 141.5953, 'confidence': 75, 'surface_temperature_celcius': 49}}
Message published successfully. Data: {'datetime': '16/02/2023 00:00:00', 'terra': {'la

Message published successfully. Data: {'datetime': '24/02/2023 00:00:00', 'terra': {'latitude': -34.5396, 'longitude': 141.7292, 'confidence': 65, 'surface_temperature_celcius': 43}}
Message published successfully. Data: {'datetime': '24/02/2023 04:48:00', 'terra': {'latitude': -38.116, 'longitude': 143.818, 'confidence': 76, 'surface_temperature_celcius': 43}}
Message published successfully. Data: {'datetime': '24/02/2023 09:36:00', 'terra': {'latitude': -36.6946, 'longitude': 144.7875, 'confidence': 100, 'surface_temperature_celcius': 117}}
Message published successfully. Data: {'datetime': '24/02/2023 14:24:00', 'terra': {'latitude': -38.1301, 'longitude': 143.9398, 'confidence': 88, 'surface_temperature_celcius': 63}}
Message published successfully. Data: {'datetime': '24/02/2023 19:12:00', 'terra': {'latitude': -37.335, 'longitude': 148.064, 'confidence': 88, 'surface_temperature_celcius': 60}}
Message published successfully. Data: {'datetime': '25/02/2023 00:00:00', 'terra': {'la

Message published successfully. Data: {'datetime': '05/03/2023 00:00:00', 'terra': {'latitude': -38.495, 'longitude': 146.944, 'confidence': 56, 'surface_temperature_celcius': 53}}
Message published successfully. Data: {'datetime': '05/03/2023 04:48:00', 'terra': {'latitude': -35.0917, 'longitude': 141.1508, 'confidence': 77, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '05/03/2023 09:36:00', 'terra': {'latitude': -37.4366, 'longitude': 143.1352, 'confidence': 74, 'surface_temperature_celcius': 48}}
Message published successfully. Data: {'datetime': '05/03/2023 14:24:00', 'terra': {'latitude': -37.8857, 'longitude': 143.2777, 'confidence': 63, 'surface_temperature_celcius': 41}}
Message published successfully. Data: {'datetime': '05/03/2023 19:12:00', 'terra': {'latitude': -37.7746, 'longitude': 148.3673, 'confidence': 88, 'surface_temperature_celcius': 65}}
Message published successfully. Data: {'datetime': '06/03/2023 00:00:00', 'terra': {'la

Message published successfully. Data: {'datetime': '14/03/2023 00:00:00', 'terra': {'latitude': -36.5104, 'longitude': 144.4674, 'confidence': 88, 'surface_temperature_celcius': 64}}
Message published successfully. Data: {'datetime': '14/03/2023 04:48:00', 'terra': {'latitude': -37.0761, 'longitude': 141.0574, 'confidence': 100, 'surface_temperature_celcius': 89}}
Message published successfully. Data: {'datetime': '14/03/2023 09:36:00', 'terra': {'latitude': -38.5233, 'longitude': 143.433, 'confidence': 61, 'surface_temperature_celcius': 41}}
Message published successfully. Data: {'datetime': '14/03/2023 14:24:00', 'terra': {'latitude': -36.3051, 'longitude': 143.1445, 'confidence': 54, 'surface_temperature_celcius': 39}}
Message published successfully. Data: {'datetime': '14/03/2023 19:12:00', 'terra': {'latitude': -36.291, 'longitude': 146.159, 'confidence': 56, 'surface_temperature_celcius': 40}}
Message published successfully. Data: {'datetime': '15/03/2023 00:00:00', 'terra': {'la

KeyboardInterrupt: 